In [1]:
import os, datetime, heapq, re
import pandas as pd
import networkx as nx

FILE_PATH = "/content/지하철 역간 시간.xlsx"

# ───────── 1) 시간 유틸 ─────────
def parse_time(timeinputdata):
    # ① 들어온 시간 데이터 확인 :  NaN → None
    if pd.isna(timeinputdata):
        return None

    # ②-① 들어온 시간 데이터가 datetime.time 객체 → 변환
    if isinstance(timeinputdata, datetime.time):
        minutes = timeinputdata.hour
        seconds = timeinputdata.minute
        return minutes*60 + seconds

    # ②-② 들어온 시간 데이터가 그 외(문자·숫자) → 문자열 변환
    text = str(timeinputdata)

    # 앞뒤 공백 수동 제거
    text = text.strip()

    # “:” 포함 → 시·분·초 형식
    if ":" in text:
        tokens = []
        for t in text.split(":"):
          tokens.append(int(t))
        # ②-② - a) 분, 초 데이터 존재 (아래 index의 경우 [0], [1]로도 표현 가능)
        if len(tokens) >= 2:
            minutes = tokens[-2]
            seconds = tokens[-1]
            return minutes*60 + seconds

    # ②-②- b) 초 데이터만 존재
    return int(float(text))


def format_sec(sec):
    """초 → 'X분 Y초' 문자열."""
    sec_round = int(round(sec))
    minutes   = sec_round // 60
    seconds   = sec_round % 60
    return f"{minutes}분 {seconds}초"

# ───────── 2) 그래프 생성용 보조 ─────────
def make_node_tuple(line, station):
    line_num = int(line)                         # 주어진 dataset 경우 해당 사항 없으나 만약 호선 번호 int 아니라면 변환
    station_clean = station.replace(" ", "")     # 주어진 dataset 경우 해당 사항 없으나 만약 역 명에 space 있을 경우
    return (line_num, station_clean)

# ───────── 3) 엑셀 → 그래프 ─────────
def build_graph(path):
    # ① 엑셀 읽기
    df = pd.read_excel(path, engine="openpyxl")

    # ② 칼럼 정리
    # 컬럼 공백 제거 – 한 줄씩 명시
    new_cols = []
    for name in df.columns:
        name = name.strip() # 양쪽(앞·뒤) 공백부터 없애기
        cleaned = "" # 글자 하나씩 살펴서 '공백'이 아니면 결과에 추가
        for ch in name:
            if not ch.isspace():      # 탭·띄어쓰기 등 모든 공백 문자 제거
                cleaned += ch
        new_cols.append(cleaned)
    df.columns = new_cols

    required = ["호선1", "역이름1", "호선2", "역이름2", "소요시간"]
    missing  = []
    for col in required:
        if col not in df.columns:
            missing.append(col)
    if missing:
        raise KeyError(f"필수 열이 없습니다: {missing}")

    # ③ 그래프 구성
    g = nx.DiGraph()

    # 각 행을 차근차근 순회
    for _, row in df.iterrows(): # index, row에서 index 사용하지 않음
        time_val = parse_time(row["소요시간"])
        if time_val is None:
            continue

        a = make_node_tuple(row["호선1"], row["역이름1"]) # vertex (2, '성수')
        b = make_node_tuple(row["호선2"], row["역이름2"]) # vertex (2, '뚝섬')

        g.add_edge(a, b, weight=time_val) # (2, '성수') --(90초)--> (2, '뚝섬')
        g.add_edge(b, a, weight=time_val) # 양방향 (2, '성수') <--(90초)-- (2, '뚝섬')
        #이동 시간(초)이 간선의 가중치(weight)로 저장됨

    return g # (출발역, 도착역, 시간정보) 하나의 간선 정보 : (2, '성수', 2, '뚝섬', 90)

# ───────── 4) 다익스트라 알고리즘 ─────────

def dijkstra_path(g: nx.DiGraph, stationstart, stationend):
    # ① 거리, 방문 정보 설정
    dist     = {stationstart: 0}     # ☆ ①-1) 각 역까지 최소 누적 시간 저장    # {(2, '성수') : 0 ,(2, '뚝섬') : 90, (2, '건대입구') : 210}
    candidates = [(0, stationstart)] # ☆ ①-2) 방문 후보 노드(아직 방문하지x) 및 누적 시간 저장 [(90, (2, '뚝섬')), (210, (2,'뚝섬입구'))]

    previous = {}                    # ①-3) 최단 경로 추적용 이전 노드 기록  # {(2, '뚝섬') : (2, '성수'), (2, '건대입구') : (2, '뚝섬;)}
    visited  = set()                 # ①-4) 경로 기록  # (2, '건대입구') 방문 확정  # {(2, '뚝섬'), (2, '성수'),(2, '건대입구')}
    transfer_count = {}              # ①-5) 역 이름별 환승 카운터 저장 (노드 기반 X)

    # ② 최단 시간 후보를 하나씩 처리
    while candidates:                            # 맨 처음에 (0, '성수')만 들어 있다
        candidates.sort()                        # 누적 시간이 가장 짧은 노드부터 처리
        acculumate_time, u = candidates.pop(0)   # accumulate_time = 90, ☆ 현재 노드 u = (2, '뚝섬') # acculumate_time = 지금까지 걸린 총 시간 # u = 현재 보는 역

        if u in visited: # 이미 방문한 역이면 패스 : visited는 중복 제거용
            continue
        visited.add(u)

        if u == stationend: # 도착역에 도달하면 멈춘다(탐색 종료)
            break

        # ③ 인접 역 확인 및 시간 갱신
        # 거쳐가는 시간 더 짧으면 다음 역들 거리 갱신, 후보 목록 추가 (지금 보는 역 u에서 갈 수 있는 역 v 검사)

        for v in g.neighbors(u):                     # ☆ v = 뚝섬, 건대입구, 용답 (v는 u와 연결된 이웃 node)

            #  환승 여부 체크
            if u[1] == v[1] and u[0] != v[0]:        # (1, '종로3가') -> (3, '종로3가') : 환승
                name = u[1]                          # '종로 3가'
                if transfer_count.get(name, 0) >= 1:
                    continue                         # 환승 1회 초과면 skip
                transfer_count[name] = 1             # 환승 1회 기록 # ①-5)

            edge_cost = g[u][v]["weight"]                        # 90 / 120 / 210 (u -> v 누적 시간 계산)
            new_accumulate_time  = acculumate_time + edge_cost   # 출발 u -> v 누적 시간 ex) 90 + 120 = 210

            # v에 도달한 적이 없거나, 더 빠른 경로를 발견할 때 update
            if v not in dist or new_accumulate_time < dist[v]: # 더 짧은 시간 발견
                dist[v] = new_accumulate_time                  # ①-1) 거리값 저장
                previous[v] = u                                # ①-4) 어디서 왔는지 저장
                candidates.append((new_accumulate_time, v))    # ①-2) update

    # ④ 경로 찾지 못한 경우
    if stationend not in dist:
        raise ValueError("경로를 찾지 못했습니다.")
    # 도착역이 dist에 없으면, 한 번도 도달하지 못했으면 에러 발생

    # ⑤ 경로 역추적 (도착 후 경로 재구성)
    path = []
    cur = stationend
    while cur != stationstart:
        path.append(cur)
        cur = previous[cur]
    path.append(stationstart)
    path.reverse()

    return dist[stationend], path  # 총 소요 시간과 최단 경로 반환

# ───────── 5) 최단 경로 (슈퍼 노드) ─────────
def shortest_path(g, start, end):
    def to_nodes(x):

        # (호선,역) 튜플이면 그대로 리스트로 감싸서 반환
        if isinstance(x, tuple):
            return [make_node_tuple(*x)]

        # 역이름만 주어지면 그래프에서 이름이 같은 노드 모두 반환    # 다중 호선 고려 가능
        result = []
        for n in g.nodes:
            if n[1] == x:
                result.append(n)
        return result

    # 출발역/도착역 후보 목록 생성
    start_candidates = to_nodes(start)    # '서울역'만 입력을 받았을 경우, start_candidates = [(1, 서울역), (4, 서울역)]
    end_candidates   = to_nodes(end)      # '교대'만 입력을 받았을 경우, end_candidates = [(2, 교대), (3, 교대)]

    # 만약 해당하는 노드가 없으면 에러 발생
    if not start_candidates or not end_candidates:
        raise ValueError("출발역 또는 도착역을 찾을 수 없습니다.")

    #슈퍼 노드 설정
    """
    모든 출발 후보를 연결하는 가상의 출발 노드(SRC)와
    모든 도착 후보를 연결하는 가상의 도착 노드(DST)를 만들어서
    출발 혹은 도착 후보를 하나의 노드로 묶는 방식
    """
    SRC = ("START", "")
    DST = ("END", "")

    #그래프에 슈퍼 노드 연결
    g.add_node(SRC)
    g.add_node(DST)

    for s in start_candidates:
        g.add_edge(SRC, s, weight=0)    # 출발 후보 각각을 가상의 출발 노드 SRC에 연결할 때, 0초로 연결
    for e in end_candidates:
        g.add_edge(e, DST, weight=0)    # 도착 후보 각각을 가상의 도착 노드 DST에 연결할 때, 0초로 연결

    #다익스트라 알고리즘 실행
    total_sec, raw_path = dijkstra_path(g, SRC, DST)

    # 그래프 원상 복구(가상 노드 삭제해서 그래프 상태 복구)
    g.remove_node(SRC)
    g.remove_node(DST)

    # START/END 제거 및 결과 경로 반환
    user_path = raw_path[1:-1]   # raw_path : [SRC, (1, 서울역), (1, 시청), ... , (1, 종로3가), DST]
    return total_sec, user_path

# ───────── 6-1) 사용자 입력 파싱 ─────────
def parse_user_station(text):
    """
    '1,광운대'  → ('1','광운대')
    '  광운대  ' → '광운대'
    """
    # 앞뒤 공백 제거 (" 광운대 " -> "광운대")
    text = text.strip()

    # 사용자가 아무것도 입력 안 했을 경우 예외 처리
    if not text:
        raise ValueError("빈 입력입니다.")

    # 쉼표가 있는지 확인
    comma_pos = text.find(",")              # 사용자 입력이 "1,광운대"라면 comma_pos(쉼표 위치)는 1, "광운대"라면 comma_pos는 -1

    # 쉼표가 있을 경우
    if comma_pos != -1:
        line_part    = text[:comma_pos]       # 쉼표를 기준으로 앞은 "1", 뒤는 "광운대"
        station_part = text[comma_pos + 1:]

        station_part = station_part.replace(" ", "")   # 역 이름 내부의 공백 제거 ("광 운 대" -> "광운대")

        if line_part.isdigit():                   # line_part가 숫자(호선 번호)로 되어 있는지 확인
            return (line_part, station_part)      # 맞다면 (호선, 역이름) 튜플로 반환

    # 쉼표가 없을 경우
    return text.replace(" ", "")      # 역 이름 내부의 공백 제거 후 문자열 그대로 반환


# ───────── 6-2) 메인 실행 ─────────
if __name__ == "__main__":
    G = build_graph(FILE_PATH)

    start_raw = input("출발역 입력 (예: 1,광운대  또는  광운대): ")
    end_raw   = input("도착역 입력 (예: 2,건대입구  또는  건대입구): ")
    start, end = parse_user_station(start_raw), parse_user_station(end_raw)

    secs, route = shortest_path(G, start, end)
    print("\n──────── 결과 ────────")
    print(f"• 출발 → 도착 : {start_raw.strip()}  →  {end_raw.strip()}")
    print(f"• 소요시간     : {format_sec(secs)}")
    print("• 경로         :", " ▶ ".join(f'{l}호선 {st}' for l, st in route))



출발역 입력 (예: 1,광운대  또는  광운대): 석계
도착역 입력 (예: 2,건대입구  또는  건대입구): 신촌

──────── 결과 ────────
• 출발 → 도착 : 석계  →  신촌
• 소요시간     : 33분 54초
• 경로         : 1호선 석계 ▶ 1호선 신이문 ▶ 1호선 외대앞 ▶ 1호선 회기 ▶ 1호선 청량리 ▶ 1호선 제기동 ▶ 1호선 신설동 ▶ 1호선 동묘앞 ▶ 1호선 동대문 ▶ 1호선 종로5가 ▶ 1호선 종로3가 ▶ 1호선 종각 ▶ 1호선 시청 ▶ 2호선 시청 ▶ 2호선 충정로 ▶ 2호선 아현 ▶ 2호선 이대 ▶ 2호선 신촌
